In [1]:
import logging
from typing import Callable, Literal, Optional, Union

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize

from bwlogger import StyleAdapter
import argparse
import logging
from pathlib import Path
from typing import Literal, Optional, Union

import numpy as np
import pandas as pd

from bwbbgdl import GoGet
from bwlogger import StyleAdapter, basic_setup
from bwutils import TODAY, Date

from utils import calculate_weights, get_available_trackers, load_trackers
from plot import plot_results

c:\Users\pcampos\Anaconda3\envs\bwgi\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
FX_TRACKER_DICT = {
    "AED": "JPFCTAED Index",
    "ARS": "JPFCTARS Index",
    "BRL": "JPFCTBRL Index",
    "CLP": "JPFCTCLP Index",
    "CNY": "JPFCTCNY Index",
    "COP": "JPFCTCOP Index",
    "CZK": "JPFCTCZK Index",
    "HUF": "JPFCTHUF Index",
    "IDR": "JPFCTIDR Index",
    "INR": "JPFCTINR Index",
    "MXN": "JPFCTMXN Index",
    "MYR": "JPFCTMYR Index",
    "PEN": "JPFCTPEN Index",
    "PHP": "JPFCTPHP Index",
    "PLN": "JPFCTPLN Index",
    "RON": "JPFCTRON Index",
    "RUB": "JPFCTRUB Index",
    "SAR": "JPFCTSAR Index",
    "THB": "JPFCTTHB Index",
    "TRY": "JPFCTTRY Index",
    "ZAR": "JPFCTZAR Index",
}
# these are in ER in USD
EM_CDS_TRACKER_DICT = {
    "BRL": "GSCDBRBE Index",
    "CNY": "GSCDCHBE Index",
    "MXN": "GSCDMEBE Index",
    "ZAR": "GSCDSOBE Index",
}

# these are in LOC ER with pnl converted to USD
IRS_TRACKER_DICT = {
    "BRL": "GSSWBRN5 Index",
    "CNY": "GSSWCNN5 Index",
    "MXN": "GSSWMXN5 Index",
    "ZAR": "GSSWZAN5 Index",
}

EQ_TRACKER_DICT = {
    "BRL": "BNPIFBR Index",  # in BRL
    "CNY": "BNPIFCNO Index",  # China onshore but with pnl converted to USD
    "ZAR": "BNPIFSA Index",  # in ZAR
    # "MXN": "???? Index",
}

In [27]:
# def country_assets_portfolio(
#     ccy: str,
#     vol_target: float = 0.1,
#     dt_ini: Date = "1990-12-31",
#     dt_end: Date = TODAY,
# ) -> pd.DataFrame:

# --------------------------------------------------------------------------------#
# ---------------------------------- PARAMETERS ----------------------------------#
# --------------------------------------------------------------------------------#

mapper_ticker = FX_TRACKER_DICT
vol_target: float = 0.1
dt_ini: Date = "1990-12-31"
dt_end: Date = TODAY

# Trading Strategy
method_weights: Literal[
    "ERC",
    "HRC",
    "IV",
] = "IV"
rebalancing_window: tuple[
    Literal["D", "W", "M", "Q", "Y"], Union[int, Literal["start", "end"]]
] = ("M", "start")

# Rebalancing Triggers
tol_by_asset: Optional[float] = None
tol_agg: Optional[float] = None

# Covariance Matrix Parameters Estimation

return_period: tuple[Literal["D", "W", "M", "Q", "Y"], int] = ("D", 21)  # period #days
return_rolling: bool = True
cov_window: Literal["expanding", "rolling"] = (
    "expanding"  # Expanding or Rolling( and period #days)
)
cov_estimate_wegihts: Optional[tuple[Literal["halflife", "alpha"], float]] = None


# --------------------------------------------------------------------------------#
# ----------------------------------- BACKTEST -----------------------------------#
# --------------------------------------------------------------------------------#
r_days: int = return_period[1]
MIN_DATA_POINTS = 252
tracker_df = load_trackers(FX_TRACKER_DICT)

backtest = pd.Series(index=tracker_df.index[MIN_DATA_POINTS + r_days:])
start_backtest = backtest.index.min()
backtest.iloc[0] = 100.0

avaialbe_trackers = get_available_trackers(
    tracker_df.iloc[: MIN_DATA_POINTS + r_days], MIN_DATA_POINTS + r_days
)
cov = (
    np.log(tracker_df).diff(r_days)[avaialbe_trackers].dropna().iloc[:MIN_DATA_POINTS].cov()
    * 252 / r_days
)
w = calculate_weights(method=method_weights, cov_matrix=cov)
adj_factor = vol_target / np.sqrt(w @ cov @ w)
w = adj_factor * w

weights_rebal = []
q = backtest.iloc[0] * w / tracker_df.loc[start_backtest]
s_rebal = q.copy()
s_rebal.name = start_backtest
weights_rebal.append(s_rebal)

for t, tm1 in zip(backtest.index[1:], backtest.index[:-1]):
    pnl = ((tracker_df.loc[t] - tracker_df.loc[tm1]) * q).sum()
    backtest[t] = backtest[tm1] + pnl

    if t.month != tm1.month:

        if tracker_df.loc[:t].shape[0] > 252:
            avaialbe_trackers = get_available_trackers(
                tracker_df.loc[:tm1], MIN_DATA_POINTS + r_days
            )
            cov = (
                np.log(tracker_df.loc[:tm1])
                .diff(r_days)[avaialbe_trackers]
                .cov()
                * 252
                / r_days
            )
        w = calculate_weights(method=method_weights, cov_matrix=cov)
        adj_factor = vol_target / np.sqrt(w @ cov @ w)
        w = adj_factor * w
        q = backtest[tm1] * w / tracker_df.loc[tm1]
        s_rebal = q.copy()
        s_rebal.name = t
        weights_rebal.append(s_rebal)

df_weights = pd.concat(weights_rebal, axis=1).T.reindex(backtest.index, method="ffill")
df_weights.columns = df_weights.columns + "_weights"
backtest = pd.concat(
    [
        tracker_df,
        df_weights,
        backtest.to_frame("assets"),
    ],
    axis=1,
    join="outer",
    sort=True,
)
backtest
# return backtest

C:\Users\pcampos\AppData\Local\Temp\ipykernel_9032\782111075.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  backtest = pd.Series(index=tracker_df.index[MIN_DATA_POINTS + r_days:])


,BRL,CLP,CNY,COP,CZK,HUF,IDR,INR,MXN,MYR,...,IDR_weights,INR_weights,MXN_weights,MYR_weights,PLN_weights,RUB_weights,THB_weights,TRY_weights,ZAR_weights,assets
date,,,,,,,,,,,,,,,,,,,,,
1999-01-04,100.000,100.000,NaN,NaN,100.000,100.000,NaN,NaN,100.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-05,100.007,100.002,NaN,NaN,99.997,99.958,NaN,NaN,100.041,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-06,100.011,100.026,NaN,NaN,99.973,99.922,NaN,NaN,100.073,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-07,99.939,99.976,NaN,NaN,99.866,99.885,NaN,NaN,99.952,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-08,99.950,100.025,NaN,NaN,99.614,99.756,NaN,NaN,100.095,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-01,285.467,92.593,108.135,141.776,130.968,158.036,245.250,149.020,231.751,180.408,...,0.076777,0.278593,0.073462,0.189688,0.079566,0.030894,0.312285,0.039595,0.061596,217.772738
2024-07-02,281.785,91.991,108.080,141.776,130.732,158.075,244.865,148.802,234.234,180.113,...,0.076777,0.278593,0.073462,0.189688,0.079566,0.030894,0.312285,0.039595,0.061596,217.235753
2024-07-03,286.142,93.049,108.045,143.092,131.632,159.079,244.591,148.779,235.295,180.080,...,0.076777,0.278593,0.073462,0.189688,0.079566,0.030894,0.312285,0.039595,0.061596,218.103129


In [24]:
tracker_df.ewm(halflife=62).cov().loc[tracker_df.index[-1]]

id,BRL,CLP,CNY,COP,CZK,HUF,IDR,INR,MXN,MYR,PLN,RUB,THB,TRY,ZAR
id,,,,,,,,,,,,,,,
BRL,126.929166,1.693236,-1.946513,33.514192,8.546998,30.811833,30.292373,-0.452477,81.787540,-1.034016,19.619401,-59.807206,12.090099,-67.110637,-20.140625
CLP,1.693236,16.371422,1.800774,-14.930708,10.107520,4.259614,12.477129,-0.592378,-13.026072,11.281717,-3.489643,-6.139082,11.629360,6.296115,0.271116
CNY,-1.946513,1.800774,1.369437,-4.745974,0.982021,-1.616057,2.832862,-0.093139,-4.978818,2.964099,-1.852477,3.250326,3.216641,2.589367,0.278451
COP,33.514192,-14.930708,-4.745974,70.356757,-4.789091,24.385835,-26.385212,2.032853,81.773199,-24.578363,34.820656,3.603937,-24.766256,-7.305170,3.721137
CZK,8.546998,10.107520,0.982021,-4.789091,8.370870,7.318444,8.484788,-0.141518,-0.752735,6.863817,2.523135,-4.916679,7.524455,1.174773,-0.156509
HUF,30.811833,4.259614,-1.616057,24.385835,7.318444,23.717932,-0.913479,0.629997,37.687464,-4.362165,19.985452,-9.847327,-2.108867,-5.102812,-0.909803
IDR,30.292373,12.477129,2.832862,-26.385212,8.484788,-0.913479,36.640047,-0.450369,-16.421878,16.835660,-11.791133,-30.201121,25.543277,-19.741468,-8.560005
INR,-0.452477,-0.592378,-0.093139,2.032853,-0.141518,0.629997,-0.450369,0.392496,1.454840,-0.816601,1.402348,0.600124,-0.401772,0.874361,0.866132
MXN,81.787540,-13.026072,-4.978818,81.773199,-0.752735,37.687464,-16.421878,1.454840,136.127901,-22.982391,45.508495,-14.998871,-23.120485,-28.200508,-5.654144


In [8]:
backtest.to_clipboard()